In [3]:
from nltk import word_tokenize
!pip3 install wordcloud

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.


In [6]:
import warnings
warnings.filterwarnings("ignore")
import pickle as pk
import pandas as pd
import wordcloud as wc
import gensim as gs
import gensim.corpora as cp
import pyLDAvis
from pyLDAvis import gensim_models
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'wordcloud'

In [ ]:
covid_date = np.datetime64('2020-04-01T01:00:00.000000+0100')
n_topics = 10

# WordCloud

In [ ]:
def word_cloud(df):
    all_words = ''
    for text in df['text']:
        all_words += ','.join(list(text)) 
        
    print('Total words {}'.format(len(all_words)))
    
    wordcloud = wc.WordCloud(background_color="white", max_words=100000, contour_width=4, scale=3, contour_color='steelblue')
    wordcloud.generate(all_words)
    return wordcloud.to_image()

# LDA

In [ ]:
def lda(df, n_topics=5):
    all_words = []
    for text in df['text']:
        all_words.append(text)
    
    # Create dictionary and corpus
    word2num = cp.Dictionary(all_words)
    texts = all_words
    
    # Get term frequency
    corpus = [word2num.doc2bow(text) for text in texts]
    
    lda_model = gs.models.LdaMulticore(corpus=corpus, id2word=word2num, num_topics=n_topics)
    doc_lda = lda_model[corpus]
    
    print('\nTopics')
    print(lda_model.print_topics())
    
    print('\nScores')
    for i in range(0, len(corpus), 500):
        for index, score in sorted(lda_model[corpus[i]], key=lambda tup: -1*tup[1]):
            print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))
    
    pyLDAvis.enable_notebook()
    return pyLDAvis.gensim.prepare(lda_model, corpus, word2num)

def lda_base(df, n_topics=5):
    all_words = []
    for text in df['text']:
        all_words.append(text)
    
    # Create dictionary and corpus
    word2num = cp.Dictionary(all_words)
    texts = all_words
    
    # Get term frequency
    corpus = [word2num.doc2bow(text) for text in texts]
    
    lda_model = gs.models.LdaMulticore(corpus=corpus, id2word=word2num, num_topics=n_topics)
    doc_lda = lda_model[corpus]
    
    print('\nTopics')
    print(lda_model.print_topics())
    
    print('\nScores')
    for i in range(0, len(corpus), 500):
        for index, score in sorted(lda_model[corpus[i]], key=lambda tup: -1*tup[1]):
            print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))
    
    pyLDAvis.enable_notebook()
    return (lda_model, corpus, word2num)

# All

In [ ]:
# Need to first get cleaned tokenized text data
uber_df_tk = pd.read_pickle(r'./data/uber_tk.p')

In [ ]:
users=set(list(uber_df_tk['source'])+list(uber_df_tk['target']))
users = [user.lower() for user in users]

In [ ]:
# from nltk.tokenize import sent_tokenize, word_tokenize
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer

# word_lemmatizer = WordNetLemmatizer()

#Get rid of handles
for i in uber_df_tk.index:
    source=uber_df_tk['source'][i].lower().replace("_","")
    target=uber_df_tk['target'][i].lower().replace("_","")
    text=uber_df_tk['text'][i]
    word_tokenize(str(text).lower().replace("'",""))
    while(source in text):
        index=text.index(source)
        del text[index]
    while(target in text):
        index=text.index(target)
        del text[index] 
    while("uber" in text):
        index=text.index("uber")
        del text[index] 
    while("httpubersupport" in text):
        index=text.index("httpubersupport")
        del text[index] 
    while("ubersupport" in text):
        index=text.index("ubersupport")
        del text[index] 
    uber_df_tk['text'][i]=text

In [ ]:
import pickle
pickle.dump(uber_df_tk,open('./data/uber_tk.p','wb'))

In [ ]:

wcloud = word_cloud(uber_df_tk)

In [ ]:
wcloud

In [ ]:
agg=pd.read_csv("./data/all_aggregated_uber_results_2.csv")

In [ ]:
alltopics=pd.DataFrame()
no_topics=10
print(corpus)
for corp in corpus:
    topic_stuff=pd.DataFrame(lda_model[corp])
    topic=topic_stuff.T.iloc[1]
    topic.index=topic_stuff.T.iloc[0]

    actual_topic=topic_stuff.sort_values(by=1).tail(1)
    actual_topic=actual_topic[0][actual_topic.index[0]]
    topic=pd.DataFrame(topic.T).T.reset_index(drop=True)
    topic['Topic']=actual_topic
    # topic=topic_stuff.T.iloc[1].append(pd.Series(actual_topic)).reset_index(drop=True)
    alltopics=alltopics.append(topic,ignore_index=True)

In [ ]:
alltopics=alltopics.fillna(0)

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
SIA = SentimentIntensityAnalyzer()
sentiments=[]
for topic in alltopics.index:
    negativity = SIA.polarity_scores(" ".join(uber_df_tk['text'][topic]))['neg']
    sentiments.append(negativity)

In [ ]:
uber_df_tk['sentiments']=sentiments

In [ ]:
convos=[]
for index in uber_df_tk.index:
    convo=" ".join(sorted([uber_df_tk['source'][index],uber_df_tk['target'][index]]))
    convos.append(convo)

In [ ]:
uber_df_tk['Convo']=convos

In [ ]:
unique_convos=set(convos)

In [ ]:
# all_sentdifferences=[]
all_sents=[]
for convo in unique_convos:
    involved=uber_df_tk[uber_df_tk['Convo']==convo].reset_index(drop=True)
#     sent=0
#     sentdifferences=[sent]
    sents=[]
    for i in involved.index:
#         sentdiff = sent-involved['sentiments'][i]
#         sent=involved['sentiments'][i]
#         sentdifferences.append(sentdiff)
        sents.append(involved['sentiments'][i])
    all_sents.append(sents)
#     all_sentdifferences.append(sentdifferences)

In [ ]:
all_sents_df=pd.DataFrame(all_sents)

In [ ]:
convo_sents=pd.DataFrame(zip(unique_convos,all_sents))

In [ ]:
convo_sents[convo_sents[0].str.contains('StlGig')]

In [ ]:
means=all_sents_df.mean()
plt.bar(means.index,means)

In [ ]:
pd.concat([uber_df_tk[['source','target','type_of_content','created_at']],alltopics],axis=1).to_csv("formatted.csv")

In [ ]:
pyLDAvis.save_html(LDAvis_prepared,'./html/all_lda_n{}.html'.format(n_topics))

# To Uber

In [ ]:
to_uber_df_tk = uber_df_tk[uber_df_tk['target'] == 'Uber_Support']

In [ ]:
to_uber_df_tk

In [ ]:
wcloud = word_cloud(to_uber_df_tk)

In [ ]:
wcloud

In [ ]:
LDAvis_prepared = lda(to_uber_df_tk, n_topics=n_topics)

In [ ]:
LDAvis_prepared[3]

In [ ]:
pyLDAvis.save_html(LDAvis_prepared,'./html/to_lda_n{}.html'.format(n_topics))

# From Uber

In [ ]:
from_uber_df_tk = uber_df_tk[uber_df_tk['source'] == 'Uber_Support']

In [ ]:
from_uber_df_tk

In [ ]:
wcloud = word_cloud(from_uber_df_tk)

In [ ]:
wcloud

In [ ]:
LDAvis_prepared = lda(from_uber_df_tk, n_topics=n_topics)

In [ ]:
pyLDAvis.save_html(LDAvis_prepared,'./html/from_lda_n{}.html'.format(n_topics))

# To Uber Pre-COVID

In [ ]:
pre_to_uber_df_tk = uber_df_tk[(uber_df_tk['target'] == 'Uber_Support') & (uber_df_tk['created_at'] <= covid_date)]

In [ ]:
pre_to_uber_df_tk

In [ ]:
wcloud = word_cloud(pre_to_uber_df_tk)

In [ ]:
wcloud

In [ ]:
LDAvis_prepared = lda(pre_to_uber_df_tk, n_topics=n_topics)

In [ ]:
LDAvis_prepared

In [ ]:
pyLDAvis.save_html(LDAvis_prepared,'./html/pre_to_lda_n{}.html'.format(n_topics))

# To Uber Post-COVID

In [ ]:
post_to_uber_df_tk = uber_df_tk[(uber_df_tk['target'] == 'Uber_Support') & (uber_df_tk['created_at'] > covid_date)]

In [ ]:
post_to_uber_df_tk

In [ ]:
wcloud = word_cloud(from_uber_df_tk)

In [ ]:
wcloud

In [ ]:
LDAvis_prepared = lda(post_to_uber_df_tk, n_topics=n_topics)

In [ ]:
LDAvis_prepared

In [ ]:
pyLDAvis.save_html(LDAvis_prepared,'./html/post_to_lda_n{}.html'.format(n_topics))

# From Uber Pre-COVID

In [ ]:
pre_from_uber_df_tk = uber_df_tk[(uber_df_tk['source'] == 'Uber_Support') & (uber_df_tk['created_at'] <= covid_date)]

In [ ]:
pre_from_uber_df_tk

In [ ]:
wcloud = word_cloud(pre_from_uber_df_tk)

In [ ]:
wcloud

In [ ]:
LDAvis_prepared = lda(pre_from_uber_df_tk, n_topics=n_topics)

In [ ]:
LDAvis_prepared

In [ ]:
pyLDAvis.save_html(LDAvis_prepared,'./html/pre_from_lda_n{}.html'.format(n_topics))

# From Uber Post-COVID

In [ ]:
post_from_uber_df_tk = uber_df_tk[(uber_df_tk['source'] == 'Uber_Support') & (uber_df_tk['created_at'] > covid_date)]

In [ ]:
post_from_uber_df_tk

In [ ]:
wcloud = word_cloud(post_from_uber_df_tk)

In [ ]:
wcloud

In [ ]:
LDAvis_prepared = lda(post_from_uber_df_tk, n_topics=n_topics)

In [ ]:
LDAvis_prepared

In [ ]:
pyLDAvis.save_html(LDAvis_prepared,'./html/post_from_lda_n{}.html'.format(n_topics))

# Pre-COVID

In [ ]:
pre_uber_df_tk = uber_df_tk[uber_df_tk['created_at'] <= covid_date]

In [ ]:
pre_uber_df_tk

In [ ]:
wcloud = word_cloud(pre_uber_df_tk)

In [ ]:
wcloud

In [ ]:
LDAvis_prepared = lda(pre_uber_df_tk, n_topics=n_topics)

In [ ]:
LDAvis_prepared

In [ ]:
pyLDAvis.save_html(LDAvis_prepared,'./html/pre_lda_n{}.html'.format(n_topics))

# Post-COVID

In [ ]:
post_uber_df_tk = uber_df_tk[uber_df_tk['created_at'] > covid_date]

In [ ]:
post_uber_df_tk

In [ ]:
wcloud = word_cloud(post_uber_df_tk)

In [ ]:
wcloud

In [ ]:
LDAvis_prepared = lda(post_uber_df_tk, n_topics=n_topics)

In [ ]:
LDAvis_prepared

In [ ]:
pyLDAvis.save_html(LDAvis_prepared,'./html/post_lda_n{}.html'.format(n_topics))